In [2]:
!pip install jiwer
!pip install ultralytics
!pip install fuzzywuzzy
import os
import cv2
import numpy as np
from PIL import Image
import torch
import torchvision.transforms as transforms
from transformers import AutoModelForSequenceClassification, AutoTokenizer, TrOCRProcessor, VisionEncoderDecoderModel
from google.colab import drive
import gdown
from ultralytics import YOLO
import re
from rapidfuzz import fuzz, process
import matplotlib.pyplot as plt
from google.colab.patches import cv2_imshow
from PIL import Image
import torch
#yoloV8 wieghts
model = YOLO("/kaggle/input/yolo-weights-word-detection/pytorch/default/1/worddetection.pt")  # Path to your downloaded weights


medicine_list=['Acetazolamide', 'Acetylcysteine', 'Albendazole', 'Amantadine', 'Amoxicillin', 'Artificial Tears', 'Atropine', 'Azithromycin', 'Bimatoprost', 'Botox', 'Brimonidine', 'Calcium Carbonate', 'Carbachol', 'Carbamazepine', 'Carbocisteine', 'Cefdinir', 'Cefixime', 'Cefuroxime', 'Cetirizine', 'Chloramphenicol', 'Chlorhexidine', 'Ciprofloxacin', 'Clindamycin', 'Collagen', 'Cyclosporine', 'Dexamethasone', 'Dextromethorphan', 'Diclofenac', 'Domperidone', 'Donepezil', 'Dorzolamide', 'Doxycycline', 'Erythromycin', 'Fluorometholone', 'Gabapentin', 'Gatifloxacin', 'Glycolic Acid', 'Guaifenesin', 'Hyaluronic Acid', 'Hydrocortisone', 'Hydroquinone', 'Hydroxyzine', 'Ibuprofen', 'Iron Supplements', 'Ketorolac', 'Lactic Acid', 'Latanoprost', 'Levetiracetam', 'Levofloxacin', 'Lidocaine', 'Lifitegrast', 'Loratadine', 'Mannitol', 'Mebendazole', 'Memantine', 'Metoclopramide', 'Metronidazole', 'Montelukast', 'Moxifloxacin', 'Multivitamins', 'Mupirocin', 'Natamycin', 'Nepafenac', 'Niacinamide', 'Nystatin', 'Ofloxacin', 'Omeprazole', 'Ondansetron', 'Oral Rehydration Salts', 'Paracetamol', 'Peptides', 'Pilocarpine', 'Pramipexole', 'Prednisolone', 'Prednisolone Acetate', 'Prednisolone Drops', 'Pregabalin', 'Probiotics', 'Ranitidine', 'Retinol', 'Rivastigmine', 'Ropinirole', 'Salbutamol', 'Salicylic Acid', 'Sodium Hyaluronate', 'Timolol', 'Tobramycin', 'Topiramate', 'Travoprost', 'Tretinoin', 'Tropicamide', 'Valproate', 'Vitamin C Serum', 'Vitamin D', 'Voriconazole', 'Zinc Sulfate', 'أتروبين', 'أزيثروميسين', 'أسيتات البريدنيزولون', 'أسيتازولاميد', 'أسيتيل سيستين', 'ألبيندازول', 'أمانتادين', 'أملاح الإماهة الفموية', 'أموكسيسيلين', 'أوفلوكساسين', 'أوميبرازول', 'أوندانسيترون', 'إريثرومايسين', 'إيبوبروفين', 'الببتيدات', 'البروبيوتيك', 'البوتوكس', 'الفيتامينات المتعددة', 'باراسيتامول', 'براميبيكسول', 'بريجابالين', 'بريدنيزولون', 'بريمونيدين', 'بيلوكاربين', 'بيماتوبروست', 'ترافوبروست', 'تروبيكاميد', 'تريتينوين', 'توبراميسين', 'توبيراميت', 'تيمولول', 'جابابنتين', 'جاتيفلوكساسين', 'حمض الجليكوليك', 'حمض الساليسيليك', 'حمض اللاكتيك', 'حمض الهيالورونيك', 'دموع صناعية', 'دورزولاميد', 'دوكسيسيكلين', 'دومبيريدون', 'دونيبيزيل', 'ديكساميثازون', 'ديكستروميثورفان', 'ديكلوفيناك', 'رانيتيدين', 'روبينيرول', 'ريتينول', 'ريفاستجمين', 'سالبيوتامول', 'سيبروفلوكساسين', 'سيتريزين', 'سيفدينير', 'سيفوريوكسيم', 'سيفيكسيم', 'سيكلوسبورين', 'غوايفينيسين', 'فالبروات', 'فلوروميثولون', 'فوريكونازول', 'فيتامين د', 'قطرات بريدنيزولون', 'كارباشول', 'كاربامازيبين', 'كاربوسيستين', 'كبريتات الزنك', 'كربونات الكالسيوم', 'كلورامفينيكول', 'كلورهيكسيدين', 'كليندامايسين', 'كولاجين', 'كيتورولاك', 'لاتانوبروست', 'لوراتادين', 'ليدوكايين', 'ليفوفلوكساسين', 'ليفيتيجراست', 'ليفيتيراسيتام', 'مانيتول', 'مصل فيتامين سي', 'مكملات الحديد', 'موبيروسين', 'موكسيفلوكساسين', 'مونتيلوكاست', 'ميبيندازول', 'ميترونيدازول', 'ميتوكلوبراميد', 'ميمانتين', 'ناتاميسين', 'نياسيناميد', 'نيبافيناك', 'نيستاتين', 'هيالورونات الصوديوم', 'هيدروكسيزين', 'هيدروكورتيزون', 'هيدروكينون']
ArabicWords=['٨', 'دقائق', '١٢', 'قبل', 'الأكل', 'أسبوع', '٥', 'النوم', 'ليلا', 'ساعات', 'لأول', 'يوم', 'مرة', 'اليقظة', '٤', 'كل', '٧', 'دقيقة', 'أثناء', '٣', '٦', 'يومين', 'الفطار', 'مرتين', '١٤', 'واحدة', 'أيام', 'الغداء', 'اللزوم', 'ساعة', 'لمدة', 'يوميا', 'بعد', 'فقط', 'صباحا', 'عند', '١٥' ,'أتروبين', 'أزيثروميسين', 'أسيتات البريدنيزولون', 'أسيتازولاميد', 'أسيتيل سيستين', 'ألبيندازول', 'أمانتادين', 'أملاح الإماهة الفموية', 'أموكسيسيلين', 'أوفلوكساسين', 'أوميبرازول', 'أوندانسيترون', 'إريثرومايسين', 'إيبوبروفين', 'الببتيدات', 'البروبيوتيك', 'البوتوكس', 'الفيتامينات المتعددة', 'باراسيتامول', 'براميبيكسول', 'بريجابالين', 'بريدنيزولون', 'بريمونيدين', 'بيلوكاربين', 'بيماتوبروست', 'ترافوبروست', 'تروبيكاميد', 'تريتينوين', 'توبراميسين', 'توبيراميت', 'تيمولول', 'جابابنتين', 'جاتيفلوكساسين', 'حمض الجليكوليك', 'حمض الساليسيليك', 'حمض اللاكتيك', 'حمض الهيالورونيك', 'دموع صناعية', 'دورزولاميد', 'دوكسيسيكلين', 'دومبيريدون', 'دونيبيزيل', 'ديكساميثازون', 'ديكستروميثورفان', 'ديكلوفيناك', 'رانيتيدين', 'روبينيرول', 'ريتينول', 'ريفاستجمين', 'سالبيوتامول', 'سيبروفلوكساسين', 'سيتريزين', 'سيفدينير', 'سيفوريوكسيم', 'سيفيكسيم', 'سيكلوسبورين', 'غوايفينيسين', 'فالبروات', 'فلوروميثولون', 'فوريكونازول', 'فيتامين د', 'قطرات بريدنيزولون', 'كارباشول', 'كاربامازيبين', 'كاربوسيستين', 'كبريتات الزنك', 'كربونات الكالسيوم', 'كلورامفينيكول', 'كلورهيكسيدين', 'كليندامايسين', 'كولاجين', 'كيتورولاك', 'لاتانوبروست', 'لوراتادين', 'ليدوكايين', 'ليفوفلوكساسين', 'ليفيتيجراست', 'ليفيتيراسيتام', 'مانيتول', 'مصل فيتامين سي', 'مكملات الحديد', 'موبيروسين', 'موكسيفلوكساسين', 'مونتيلوكاست', 'ميبيندازول', 'ميترونيدازول', 'ميتوكلوبراميد', 'ميمانتين', 'ناتاميسين', 'نياسيناميد', 'نيبافيناك', 'نيستاتين', 'هيالورونات الصوديوم', 'هيدروكسيزين', 'هيدروكورتيزون', 'هيدروكينون']
EnglishWords=['Acetazolamide', 'Acetylcysteine', 'Albendazole', 'Amantadine', 'Amoxicillin', 'Artificial Tears', 'Atropine', 'Azithromycin', 'Bimatoprost', 'Botox', 'Brimonidine', 'Calcium Carbonate', 'Carbachol', 'Carbamazepine', 'Carbocisteine', 'Cefdinir', 'Cefixime', 'Cefuroxime', 'Cetirizine', 'Chloramphenicol', 'Chlorhexidine', 'Ciprofloxacin', 'Clindamycin', 'Collagen', 'Cyclosporine', 'Dexamethasone', 'Dextromethorphan', 'Diclofenac', 'Domperidone', 'Donepezil', 'Dorzolamide', 'Doxycycline', 'Erythromycin', 'Fluorometholone', 'Gabapentin', 'Gatifloxacin', 'Glycolic Acid', 'Guaifenesin', 'Hyaluronic Acid', 'Hydrocortisone', 'Hydroquinone', 'Hydroxyzine', 'Ibuprofen', 'Iron Supplements', 'Ketorolac', 'Lactic Acid', 'Latanoprost', 'Levetiracetam', 'Levofloxacin', 'Lidocaine', 'Lifitegrast', 'Loratadine', 'Mannitol', 'Mebendazole', 'Memantine', 'Metoclopramide', 'Metronidazole', 'Montelukast', 'Moxifloxacin', 'Multivitamins', 'Mupirocin', 'Natamycin', 'Nepafenac', 'Niacinamide', 'Nystatin', 'Ofloxacin', 'Omeprazole', 'Ondansetron', 'Oral Rehydration Salts', 'Paracetamol', 'Peptides', 'Pilocarpine', 'Pramipexole', 'Prednisolone', 'Prednisolone Acetate', 'Prednisolone Drops', 'Pregabalin', 'Probiotics', 'Ranitidine', 'Retinol', 'Rivastigmine', 'Ropinirole', 'Salbutamol', 'Salicylic Acid', 'Sodium Hyaluronate', 'Timolol', 'Tobramycin', 'Topiramate', 'Travoprost', 'Tretinoin', 'Tropicamide', 'Valproate', 'Vitamin C Serum', 'Vitamin D', 'Voriconazole', 'Zinc Sulfate''for', 'Every', 'After', 'Twice', 'needed', 'first', 'hours', 'breakfast', 'For', '8', 'awake', 'other', 'Night', 'hour', 'Morning', 'bed', '14', 'the', 'days', 'As', '5', 'bedtime', 'lunch', 'week', '7', 'daily', 'only', '4', 'Before', 'while', '3', 'minutes', '2', '6', 'day', '15', 'meals', 'Once', '12']
frequency_patterns = [
    ## 🔹 English Frequencies
    r"\bevery\s\d+\shours?\b",            # "every 6 hours"
    r"\bEvery\shours\b",
    r"\bevery\s\d+\s?-\s?\d+\shours?\b",  # "every 4-6 hours"
    r"\bonce\sdaily\b",                   # "once daily"
    r"\btwice\sdaily\b",                  # "twice daily"
    r"\bthree\stimes\sa\sday\b",          # "three times a day"
    r"\bfour\stimes\sa\sday\b",           # "four times a day"
    r"\b\d+\stimes\sa\sday\b",            # "5 times a day"

    ## ⏳ Time-Based
    r"\bevery\sother\sday\b",             # "every other day"
    r"\bevery\s\d+\sdays?\b",             # "every 3 days"
    r"\bevery\s\d+\sweeks?\b",            # "every 2 weeks"
    r"\bevery\s\d+\smonths?\b",           # "every 6 months"

    ## 🌙 Morning/Evening
    r"\bin\sthe\smorning\b",              # "in the morning"
    r"\bin\sthe\sevening\b",              # "in the evening"
    r"\bin\sthe\safternoon\b",            # "in the afternoon"
    r"\bin\sthe\snight\b",                # "in the night"
    r"\bdaily\sat\snoon\b",               # "daily at noon"

    ## 🍽 Meal-Based
    r"\bbefore\smeals?\b",                # "before meals"
    r"\bbefore\sbreakfast?\b",            # "before breakfast"
    r"\bafter\smeals?\b",                 # "after meals"
    r"\bafter\sbreakfast?\b",             # "after breakfast"
    r"\bbefore\sfood\b",                  # "before food"
    r"\bafter\sfood\b",                   # "after food"
    r"\bon\san\sempty\sstomach\b",        # "on an empty stomach"

    ## 🌙 Sleep
    r"\bbefore\sbedtime\b",               # "before bedtime"
    r"\bat\sbedtime\b",                   # "at bedtime"
    r"\bbefore\sgoing\sto\sbed\b",        # "before going to bed"

    ## 🔄 PRN (As Needed)
    r"\bas\sneeded\b",                    # "as needed"
    r"\bif\sneeded\b",                    # "if needed"
    r"\bwhen\snecessary\b",               # "when necessary"
    r"\bwhen\srequired\b",                # "when required"
    r"\bwhen\sfeeling\spain\b",           # "when feeling pain"

    ## 🚑 Perioperative
    r"\bbefore\ssurgery\b",               # "before surgery"
    r"\bafter\ssurgery\b",                # "after surgery"
    r"\bbefore\san\soperation\b",         # "before an operation"
    r"\bafter\san\soperation\b",          # "after an operation"

    ## 🔹 Arabic Frequencies
    r"\bكل\s\d+\sساعة\b",              # "كل 8 ساعات" (every X hours)
    r"\bكل\s\d+\s?-\s?\d+\sساعات?\b",  # "كل 4-6 ساعات" (every X-Y hours)
    r"\bمرة\sيوميا\b",                 # "مرة يوميًا" (once daily)
    r"\bمرة\sكل\sيوم\b",               # "مرة كل يوم" (once per day)
    r"\bمرة\sيوميا\b",                 # "مرة يوميا" (once per day)
    r"\bمرة\sأسبوعيا\b",               # "مرة أسبوعيًا" (once weekly)
    r"\bمرة\sكل\sأسبوع\b",             # "مرة كل أسبوع" (once per week)
    r"\bمرة\sشهريا\b",                 # "مرة شهريًا" (once monthly)
    r"\bمرة\sكل\sشهر\b",               # "مرة كل شهر" (once per month)
    r"\bمرتين\sيوميا\b",               # "مرتين يوميًا" (twice daily)
    r"\b\d+\sمرات?\sيوميا\b",          # "3 مرات يوميًا" (multiple times daily)

    ## ⏳ Time-Based
    r"\bكل\sيومين\b",                   # "كل يومين" (every other day)
    r"\bكل\s\d+\sأيام\b",               # "كل 3 أيام" (every X days)
    r"\bكل\s\d+\sأسابيع\b",             # "كل 2 أسابيع" (every X weeks)
    r"\bكل\s\d+\sشهور\b",               # "كل 6 شهور" (every X months)

   ## Arabic (Word-Based Numbers)
    r"\bكل\sساعة\b",                     # "كل واحدة ساعة" (every one hour)
    r"\bكل\sساعتين\b",                   # "كل اثنتين ساعة" (every two hours)
    r"\bكل\sثلاث\sساعات\b",               # "كل ثلاث ساعات" (every three hours)
    r"\bكل\sأربع\sساعات\b",              # "كل أربع ساعات" (every four hours)
    r"\bكل\sخمس\sساعات\b",               # "كل خمس ساعات" (every five hours)
    r"\bكل\sست\sساعات\b",                # "كل ست ساعات" (every six hours)
    r"\bكل\sسبع\sساعات\b",               # "كل سبع ساعات" (every seven hours)
    r"\bكل\sثماني\sساعات\b",             # "كل ثماني ساعات" (every eight hours)
    r"\bكل\sتسع\sساعات\b",               # "كل تسع ساعات" (every nine hours)
    r"\bكل\sعشر\sساعات\b",               # "كل عشر ساعات" (every ten hours)
    r"\bكل\sإحدى\sعشرة\sساعة\b",         # "كل إحدى عشرة ساعة" (every 11 hours)
    r"\bكل\sاثنتي\sعشرة\sساعة\b",        # "كل اثنتي عشرة ساعة" (every 12 hours)

    ## 🌙 Morning/Evening
    r"\bفي\sالصباح\b",                  # "في الصباح" (in the morning)
    r"\bفي\sالمساء\b",                  # "في المساء" (in the evening)
    r"\bفي\sالظهيرة\b",                 # "في الظهيرة" (at noon)
    r"\bفي\sالليل\b",                   # "في الليل" (at night)

    ## 🍽 Meal-Based
    r"\bقبل\sالأكل\b",                  # "قبل الأكل" (before meals)
    r"\bبعد\sالأكل\b",                  # "بعد الأكل" (after meals)
    r"\bقبل\sالطعام\b",                 # "قبل الطعام" (before food)
    r"\bبعد\sالطعام\b",                 # "بعد الطعام" (after food)
    r"\bعلى\sمعدة\sفارغة\b",            # "على معدة فارغة" (on an empty stomach)
    r"\bعلى\sالريق\b",                   # "على الريق" (fasting)

    ## 🌙 Sleep
    r"\bقبل\sالنوم\b",                 # "قبل النوم" (before sleep)
    r"\bعند\sالنوم\b",                 # "عند النوم" (at bedtime)

    ## 🔄 PRN (As Needed)
    r"\bعند\sاللزوم\b",                # "عند اللزوم" (as needed)
    r"\bحسب\sالحاجة\b",                # "حسب الحاجة" (as required)
    r"\bإذا\sاستدعت\sالحاجة\b",         # "إذا استدعت الحاجة" (if necessary)
    r"\bعند\sالشعور\sبالألم\b",          # "عند الشعور بالألم" (when feeling pain)

    ## 🚑 Perioperative
    r"\bقبل\sالعملية\b",              # "قبل العملية" (before surgery)
    r"\bبعد\sالعملية\b",              # "بعد العملية" (after surgery)
    r"\bقبل\sالتدخل\sالجراحي\b",     # "قبل التدخل الجراحي" (before an operation)
    r"\bبعد\sالتدخل\sالجراحي\b",     # "بعد التدخل الجراحي" (after an operation)
    r"\bEvery\s\d+\shours?\b",            # "Every 8 hours" (capitalized)
    r"\bEvery\shours?\b",                 # "Every hours" (capitalized)
    r"\bAfter\slunch\b",                  # "After lunch" (capitalized)
    r"\bOnce\sdaily\b",                   # "Once daily" (capitalized)

]



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 46.0 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 974.8/974.8 kB 20.3 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.9 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.2 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 28.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 1.9 MB/s eta 0:00:00:00:0100:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 53.7 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.8.93
    Uninstalling nvidia-nvjitlink-cu12-12.8.93:
      Successfully uninstalled n

2025-04-16 17:58:08.882424: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1744826289.132199      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1744826289.210441      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [3]:
MODEL_NAME = "microsoft/trocr-base-handwritten"
processor = TrOCRProcessor.from_pretrained(MODEL_NAME) # Initialize the processor here
new_model = VisionEncoderDecoderModel.from_pretrained(MODEL_NAME)
new_model.config.decoder_start_token_id = processor.tokenizer.cls_token_id
new_model.config.pad_token_id = processor.tokenizer.pad_token_id
new_model.load_state_dict(torch.load("/kaggle/input/trocr_finetune_weights_stp/pytorch/default/1/model_weights.pth"))
DEFAULT_FREQUENCY = "Every 6 hours"

preprocessor_config.json:   0%|          | 0.00/224 [00:00<?, ?B/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


tokenizer_config.json:   0%|          | 0.00/1.12k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/4.17k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.33G [00:00<?, ?B/s]

Config of the encoder: <class 'transformers.models.vit.modeling_vit.ViTModel'> is overwritten by shared encoder config: ViTConfig {
  "attention_probs_dropout_prob": 0.0,
  "encoder_stride": 16,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.0,
  "hidden_size": 768,
  "image_size": 384,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "model_type": "vit",
  "num_attention_heads": 12,
  "num_channels": 3,
  "num_hidden_layers": 12,
  "patch_size": 16,
  "pooler_act": "tanh",
  "pooler_output_size": 768,
  "qkv_bias": false,
  "torch_dtype": "float32",
  "transformers_version": "4.51.1"
}

Config of the decoder: <class 'transformers.models.trocr.modeling_trocr.TrOCRForCausalLM'> is overwritten by shared decoder config: TrOCRConfig {
  "activation_dropout": 0.0,
  "activation_function": "gelu",
  "add_cross_attention": true,
  "attention_dropout": 0.0,
  "bos_token_id": 0,
  "classifier_dropout": 0.0,
  "cross_attention_hidden_size": 768,
  "d_mod

generation_config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [7]:
from sklearn.cluster import DBSCAN

def preprocess_for_trocr(cropped_image):
    # Convert BGR (OpenCV) to RGB (PIL)
    rgb_image = cv2.cvtColor(cropped_image, cv2.COLOR_BGR2RGB)
    pil_image = Image.fromarray(rgb_image)
    return pil_image

def fuzzy_match(word, word_list, threshold=30):
    result = process.extractOne(word, word_list, scorer=fuzz.ratio)
    if result:
        # Handle different versions of fuzzywuzzy
        if isinstance(result, tuple) and len(result) >= 2:
            match, score = result[0], result[1]  # Extract match and score
            return match if score >= threshold else None
    return None

def remove_duplicate_boxes(boxes, iou_threshold=0.8):
    if len(boxes) == 0:
        return []

    # Convert boxes to (x1, y1, x2, y2) format
    boxes = np.array(boxes)
    x1 = boxes[:, 0]
    y1 = boxes[:, 1]
    x2 = boxes[:, 2]
    y2 = boxes[:, 3]
    # Compute the area of the bounding boxes
    areas = (x2 - x1 + 1) * (y2 - y1 + 1)

    # Sort the boxes by the bottom-right y-coordinate
    indices = np.argsort(y2)
    keep = []

    while len(indices) > 0:
        last = len(indices) - 1
        i = indices[last]
        keep.append(i)

        # Compute the IoU of the remaining boxes with the last box
        xx1 = np.maximum(x1[i], x1[indices[:last]])
        yy1 = np.maximum(y1[i], y1[indices[:last]])
        xx2 = np.minimum(x2[i], x2[indices[:last]])
        yy2 = np.minimum(y2[i], y2[indices[:last]])

        w = np.maximum(0, xx2 - xx1 + 1)
        h = np.maximum(0, yy2 - yy1 + 1)
        overlap = (w * h) / areas[indices[:last]]

        # Remove boxes with IoU greater than the threshold
        indices = np.delete(indices, np.concatenate(([last], np.where(overlap > iou_threshold)[0])))

    return boxes[keep].tolist()

def group_boxes_into_lines_and_sort(boxes, texts, vertical_threshold=20):
    if not boxes or not texts:
        return [], []
    
    # Convert boxes to include center points and other useful metrics
    box_data = []
    for i, (box, text) in enumerate(zip(boxes, texts)):
        if text:  # Only process boxes with valid text
            x1, y1, x2, y2 = box
            center_y = (y1 + y2) / 2
            center_x = (x1 + x2) / 2
            height = y2 - y1
            box_data.append({
                'index': i,
                'box': box,
                'text': text,
                'center_x': center_x,
                'center_y': center_y,
                'height': height
            })
    
    if not box_data:
        return [], []
    
    # Dynamically adjust vertical threshold based on average text height
    avg_height = sum(item['height'] for item in box_data) / len(box_data)
    dynamic_threshold = max(vertical_threshold, avg_height * 0.7)
    
    # Initial sort by y-coordinate
    box_data.sort(key=lambda x: x['center_y'])
    
    # Use DBSCAN clustering to group lines
    points = np.array([[item['center_x'], item['center_y']] for item in box_data])
    # Adjust eps based on average height
    clustering = DBSCAN(eps=dynamic_threshold, min_samples=1).fit(points)
    labels = clustering.labels_
    
    # Group boxes by cluster label
    lines = {}
    for i, label in enumerate(labels):
        if label not in lines:
            lines[label] = []
        lines[label].append(box_data[i])
    
    # Sort lines by average y-coordinate
    sorted_lines = sorted(lines.values(), key=lambda line: sum(item['center_y'] for item in line)/len(line))
    
    # Sort each line by x-coordinate (left to right)
    for line in sorted_lines:
        line.sort(key=lambda x: x['center_x'])
    
    # Extract the ordered text and boxes
    ordered_text = []
    ordered_boxes = []
    for line in sorted_lines:
        for box in line:
            ordered_text.append(box['text'])
            ordered_boxes.append(box['box'])
    
    return ordered_text, ordered_boxes

def TextExtraction(images):
    extracted_data = []
    for image_path in images:
        image = cv2.imread(image_path)
        image_data = {
            "image_path": image_path,
            "text": "",  # Full text in reading order
            "words": []  # List to store {text, position} for each word
        }
        
        # Run YOLOv8 detection
        results = model(image)
        boxes = results[0].boxes.xyxy.cpu().numpy()
        class_ids = results[0].boxes.cls.cpu().numpy()
        
        # Remove duplicate boxes
        boxes = remove_duplicate_boxes(boxes)
        
        # Extract text from each box
        all_texts = []
        for i, box in enumerate(boxes):
            x1, y1, x2, y2 = map(int, box)
            cropped_image = image[y1:y2, x1:x2]
            
            # Preprocess for TrOCR
            trocr_input = preprocess_for_trocr(cropped_image)
            inputs = processor(images=trocr_input, return_tensors="pt").to(new_model.device)
            
            # Extract text
            with torch.no_grad():
                generated_ids = new_model.generate(**inputs)
                output_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
            
            # Fuzzy match with language-specific dictionary
            class_id = int(class_ids[i])
            word_list = ArabicWords if class_id == 0 else EnglishWords
            matched_text = fuzzy_match(output_text.strip(), word_list)
            if matched_text:
                all_texts.append(matched_text)
                # Save the original box for later
                boxes[i] = box  # Ensure box is in the right format
        
        # Group into lines and sort in reading order
        ordered_texts, ordered_boxes = group_boxes_into_lines_and_sort(boxes, all_texts)
        
        # Build the complete text and word objects
        full_text = " ".join(ordered_texts)
        image_data["text"] = full_text
        
        for text, box in zip(ordered_texts, ordered_boxes):
            x1, y1, x2, y2 = map(int, box)
            image_data["words"].append({
                "text": text,
                "position": (x1, y1, x2, y2)
            })
        
        extracted_data.append(image_data)
    
    return extracted_data

def sort_words_by_position(word_data):
    # Sort by y1 (top-to-bottom), then by x1 (left-to-right)
    return sorted(word_data, key=lambda x: (x["position"][1], x["position"][0]))

def print_sorted_words(sorted_words):
    print("Sorted Words (Top-to-Bottom, Left-to-Right):")
    for i, word in enumerate(sorted_words):
        print(f"{i+1}: '{word['text']}' at {word['position']}")

def organize_medication_instructions(words_with_coords):
    # Step 1: Calculate the center y-coordinate for each word's bounding box
    words_with_y_center = []
    for word_dict in words_with_coords:
        word = word_dict['text']
        coords = word_dict['position']  # Assuming this is (x1, y1, x2, y2)
        x1, y1, x2, y2 = coords
        y_center = (y1 + y2) / 2
        x_center = (x1 + x2) / 2
        words_with_y_center.append((word, y_center, x_center, coords))
    
    # Step 2: Sort words by their y-coordinate (top to bottom)
    words_with_y_center.sort(key=lambda x: x[1])
    
    # Step 3: Group words that are likely on the same line
    # We'll consider words to be on the same line if their y-centers are within a threshold
    y_threshold = 100  # This can be adjusted based on your document's layout
    
    lines = []
    current_line = [words_with_y_center[0]]
    
    for i in range(1, len(words_with_y_center)):
        current_word = words_with_y_center[i]
        previous_word = words_with_y_center[i-1]
        
        # If this word is close in y-position to the previous word, add it to the current line
        if abs(current_word[1] - previous_word[1]) < y_threshold:
            current_line.append(current_word)
        # Otherwise, start a new line
        else:
            lines.append(current_line)
            current_line = [current_word]
    
    # Add the last line if it's not empty
    if current_line:
        lines.append(current_line)
    
    # Step 4: Sort words within each line by x-coordinate (left to right)
    for i in range(len(lines)):
        lines[i].sort(key=lambda x: x[2])  # Sort by x_center
    
    # Step 5: Join words in each line to form complete instructions
    instructions = []
    for line in lines:
        instruction = " ".join(word[0] for word in line)
        instructions.append(instruction)
    
    return instructions

def create_medication_frequency_pairs(organized_instructions):
    # Step 1: Concatenate all instructions into one line
    all_text = " ".join(organized_instructions)
    
    
    # Step 3: Extract medication-frequency pairs
    medication_pairs = []
    
    # Simple approach: Find a medication and assume text until the next medication is its frequency
    for i, med in enumerate(medicine_list):
        match = re.search(r'\b' + med + r'\b', all_text, re.IGNORECASE)
        if match:
            start_idx = match.start()
            
            # Find the next medication in the text after this one
            next_med_idx = float('inf')
            for other_med in medicine_list:
                if other_med != med:
                    other_match = re.search(r'\b' + other_med + r'\b', all_text[start_idx+len(med):], re.IGNORECASE)
                    if other_match:
                        if start_idx + len(med) + other_match.start() < next_med_idx:
                            next_med_idx = start_idx + len(med) + other_match.start()
            
            # Extract the frequency instruction
            if next_med_idx != float('inf'):
                frequency = all_text[start_idx+len(med):next_med_idx].strip()
            else:
                frequency = all_text[start_idx+len(med):].strip()
            
            medication_pairs.append((med, frequency))
    
    # Step 4: More sophisticated approach using patterns specific to medication instructions
    if not medication_pairs:
        # Pattern: Look for medication name followed by frequency info
        patterns = [
            # Medication followed by "Every X hours"
            (r'(\b\w+\b)\s+Every\s+(\d+)\s+hours', lambda m: (m.group(1), f"Every {m.group(2)} hours")),
            
            # Medication followed by "Once daily"
            (r'(\b\w+\b)\s+Once\s+daily', lambda m: (m.group(1), "Once daily")),
            
            # Medication followed by "After lunch"
            (r'(\b\w+\b)\s+After\s+lunch', lambda m: (m.group(1), "After lunch")),
            
            # More patterns can be added for different frequency formats
        ]
        
        for pattern, extract_func in patterns:
            for match in re.finditer(pattern, all_text, re.IGNORECASE):
                medication_pairs.append(extract_func(match))
    
    return medication_pairs


In [10]:
def submit(images):
    results = [] 
    extracted_data = TextExtraction(images)

    for prescription_data in extracted_data:
        sorted_words = sort_words_by_position(prescription_data["words"])
        print(f"\nSorted words for {prescription_data['image_path']}:")
        print_sorted_words(sorted_words)
        print("-------------------------------------------------------------------------------")
        organized_instructions = organize_medication_instructions(sorted_words)

        print("Organized Medication Instructions:")
        for i, instruction in enumerate(organized_instructions, 1):
            print(f"{i}. {instruction}")
        print("-------------------------------------------------------------------------------")
        medication_pairs = create_medication_frequency_pairs(organized_instructions)

        print("\nMedication-Frequency Pairs:")
        for med, freq in medication_pairs:
            print(f"- {med}: {freq}")
        results.append({
            "image_path": prescription_data["image_path"],
            "prescription_pairs": medication_pairs
        })

    return results

In [ ]:
# Example list of image file paths (update with actual paths)
images = ['/kaggle/input/machathon6-phase1-images/Beauty_prescription_104.jpg','/kaggle/input/machathon6-phase1-images/Beauty_prescription_116.jpg','/kaggle/input/machathon6-phase1-images/Dental_prescription_591.jpg', '/kaggle/input/machathon6-phase1-images/Neurology_prescription_131.jpg']

# Call the submit function
results = submit(images)

# Print the structured results
for i, result in enumerate(results):
    print(f"\nPrescription {i + 1} ({result['image_path']}):")
    if not result["prescription_pairs"]:
        print("  No medicine-frequency pairs found")
    else:
        for medicine, frequency in result["prescription_pairs"]:
            print(f"  Medicine: {medicine}, Frequency: {frequency}")